In [1]:
# Insert code here.
import pandas as pd
import numpy as np
import random
import re
import time
import datetime
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, BertConfig, AutoModel
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from torch.utils.data import Dataset
from tqdm import tqdm

# from sentence_transformers import SentenceTransformer
# sent_encoder = SentenceTransformer('bert-base-nli-mean-tokens')

In [2]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:0")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
torch.cuda.empty_cache()

There are 4 GPU(s) available.
We will use the GPU: GeForce RTX 2080 Ti


In [3]:
models = ['bert-base-multilingual-cased', 'xlm-roberta-base', 'sagorsarker/bangla-bert-base', 'ai4bharat/indic-bert', '/scratch/indic-tapt/checkpoint-5000/']
model_num = 4
tokenizer = AutoTokenizer.from_pretrained(models[model_num])


In [4]:
train = pd.read_csv('data/train.tsv', sep='\t')
test = pd.read_csv('data/valid.tsv', sep='\t')

In [5]:
train.sample(20)

,Unique ID,Post,Labels Set
125,126,"बिहार: समान काम, समान वेतन की मांग को लेकर प्र...",non-hostile
4775,4977,कोरोना महामारी के कारण देश भर में जनता परेशान ...,non-hostile
977,978,"लोकसभा ने बनाया इतिहास, पिछले 20 साल में सबसे ...",non-hostile
2731,2847,#Noida में अब तक 145 पुलिसकर्मी #कोरोना_पॉजिटि...,non-hostile
5341,5543,कश्मीर की जन्नत को जहन्नम की आग बनाने वालों इस...,offensive
3310,3426,मुसलमानों को ज्ञान देता हुआ एक मुसलमान बुद्धिज...,"hate,offensive"
2080,2148,भाईया भारतीय मुस्लिम भाईयों आप सभी देश के वामप...,offensive
277,278,5 महीने हो गए इन साधु की हत्या हुई .. आज तक को...,hate
3148,3264,India-China Rift: लद्दाख में BRO ने लगाई नई मश...,non-hostile
2419,2535,"गलती मोदी की नहीं जो देश को धोका दिया, गलती उन...",fake


In [6]:
def encode_labels(label):
    tmp = label.split(',')
    ls = [0, 0, 0, 0]
    if tmp[0] == 'non-hostile':
        return ls
    if 'fake' in tmp:
        ls[0] = 1
    if 'hate' in tmp:
        ls[1] = 1
    if 'offensive' in tmp:
        ls[2] = 1
    if 'defamation' in tmp:
        ls[3] = 1
    return ls

In [7]:
train['encodelabels'] = train['Labels Set'].apply(encode_labels)

In [8]:
train.sample(20)

,Unique ID,Post,Labels Set,encodelabels
4276,4392,पूर्व राष्ट्रपति #PranabMukherjee के राजनीतिक ...,non-hostile,"[0, 0, 0, 0]"
3132,3248,RT @Rofl_RavishNDTV: ब्रेकिंग न्यूज़- कानपुर ...,hate,"[0, 1, 0, 0]"
3831,3947,#Sitapur पेट्रोल पंप पर 2 दबंगों ने की सेल्समै...,non-hostile,"[0, 0, 0, 0]"
2088,2156,सुने हैं कि करण जौहर गे है... गे का मतलब होता ...,defamation,"[0, 0, 0, 1]"
3035,3151,@ArvindD68203391 एकदम सही बात बोले आप ये साले ...,hate,"[0, 1, 0, 0]"
1085,1086,@bewda_anna @Rolf_007 IPL देख रहा होगा कमिना,offensive,"[0, 0, 1, 0]"
4999,5201,"रिलायंस रिटेल को मिल सकता है बड़ा निवेश, वॉलमा...",non-hostile,"[0, 0, 0, 0]"
3280,3396,#SureshRaina के ट्वीट के बाद पंजाब पुलिस का बड...,non-hostile,"[0, 0, 0, 0]"
4972,5174,Motorola One 5G है कंपनी का किफायती 5जी स्मार्...,non-hostile,"[0, 0, 0, 0]"
4860,5062,ये अवैध मानव अंग तस्करी का धंधा है। जो कोरोना ...,non-hostile,"[0, 0, 0, 0]"


In [9]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [10]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-05


In [11]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Post
        self.targets = self.data.encodelabels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_attention_mask = True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [12]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_data=train.sample(frac=train_size,random_state=200)
test_data=train.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(train.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (5527, 4)
TRAIN Dataset: (4422, 4)
TEST Dataset: (1105, 4)


In [13]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [14]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained(models[model_num])
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
             

In [15]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [16]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [17]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
#         if _%50==0:
#             print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    print(f'Epoch: {epoch}, Loss:  {loss.item()}')
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    final_outputs = np.array(fin_outputs) >=0.5
    final = []
    final_t = []
    final_fine = [[],[],[],[]]
    final_fine_t = [[],[],[],[]]
    for (i,j) in zip(final_outputs, fin_targets):
        output_sum = sum(i)
        target_sum = sum(j)
        if output_sum == 0:
            final.append(0)
        else:
            final.append(1)
        if target_sum == 0:
            final_t.append(0)
        else:
            final_t.append(1)
        for p in range(4):
            final_fine[p].append(int(i[p]))
            final_fine_t[p].append(int(j[p]))
    print("Coarse:")
    print(classification_report(final, final_t))
    for i in range(4):
        print("Fine", i)
        print(classification_report(final_fine[i], final_fine_t[i]))
#     return fin_outputs, fin_targets

In [18]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]/home/tathagata.raha/miniconda3/envs/fastai/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1764: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
139it [00:33,  4.09it/s]
2it [00:00, 10.73it/s]

Epoch: 0, Loss:  0.3956594467163086


35it [00:03, 10.64it/s]
/home/tathagata.raha/miniconda3/envs/fastai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
0it [00:00, ?it/s]

Coarse:
              precision    recall  f1-score   support

           0       1.00      0.52      0.69      1105
           1       0.00      0.00      0.00         0

    accuracy                           0.52      1105
   macro avg       0.50      0.26      0.34      1105
weighted avg       1.00      0.52      0.69      1105

Fine 0
              precision    recall  f1-score   support

           0       1.00      0.79      0.88      1105
           1       0.00      0.00      0.00         0

    accuracy                           0.79      1105
   macro avg       0.50      0.40      0.44      1105
weighted avg       1.00      0.79      0.88      1105

Fine 1
              precision    recall  f1-score   support

           0       1.00      0.86      0.93      1105
           1       0.00      0.00      0.00         0

    accuracy                           0.86      1105
   macro avg       0.50      0.43      0.46      1105
weighted avg       1.00      0.86      0.93      110

139it [00:34,  4.07it/s]
2it [00:00, 10.53it/s]

...

Coarse:
              precision    recall  f1-score   support

           0       0.98      0.78      0.87       725
           1       0.69      0.97      0.81       380

    accuracy                           0.84      1105
   macro avg       0.84      0.87      0.84      1105
weighted avg       0.88      0.84      0.85      1105

Fine 0
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       879
           1       0.79      0.81      0.80       226

    accuracy                           0.92      1105
   macro avg       0.87      0.88      0.87      1105
weighted avg       0.92      0.92      0.92      1105

Fine 1
              precision    recall  f1-score   support

           0       0.94      0.89      0.92      1002
           1       0.31      0.47      0.37       103

    accuracy                           0.85      1105
   macro avg       0.63      0.68      0.65      1105
weighted avg       0.88      0.85      0.87      110

139it [00:35,  3.95it/s]
2it [00:00, 10.19it/s]

Epoch: 9, Loss:  0.11809884011745453


35it [00:03, 10.25it/s]

Coarse:
              precision    recall  f1-score   support

           0       0.98      0.78      0.87       719
           1       0.71      0.97      0.82       386

    accuracy                           0.85      1105
   macro avg       0.84      0.87      0.84      1105
weighted avg       0.88      0.85      0.85      1105

Fine 0
              precision    recall  f1-score   support

           0       0.96      0.93      0.94       911
           1       0.71      0.84      0.77       194

    accuracy                           0.91      1105
   macro avg       0.84      0.88      0.86      1105
weighted avg       0.92      0.91      0.91      1105

Fine 1
              precision    recall  f1-score   support

           0       0.92      0.90      0.91       974
           1       0.38      0.45      0.41       131

    accuracy                           0.85      1105
   macro avg       0.65      0.68      0.66      1105
weighted avg       0.86      0.85      0.85      110

In [19]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [20]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

35it [00:03, 10.35it/s]


In [21]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")

Hamming Score = 0.7239819004524887
Hamming Loss = 0.1167420814479638


In [22]:
from sklearn.metrics import classification_report
total = len(targets)
final = []
final_t = []
final_fine = [[],[],[],[]]
final_fine_t = [[],[],[],[]]
for (i,j) in zip(final_outputs, targets):
    output_sum = sum(i)
    target_sum = sum(j)
    if output_sum == 0:
        final.append(0)
    else:
        final.append(1)
    if target_sum == 0:
        final_t.append(0)
    else:
        final_t.append(1)
    for p in range(4):
        final_fine[p].append(int(i[p]))
        final_fine_t[p].append(int(j[p]))
print("Coarse:")
print(classification_report(final, final_t))
for i in range(4):
    print("Fine", i)
    print(classification_report(final_fine[i], final_fine_t[i]))

Coarse:
              precision    recall  f1-score   support

           0       0.98      0.78      0.87       719
           1       0.71      0.97      0.82       386

    accuracy                           0.85      1105
   macro avg       0.84      0.87      0.84      1105
weighted avg       0.88      0.85      0.85      1105

Fine 0
              precision    recall  f1-score   support

           0       0.96      0.93      0.94       911
           1       0.71      0.84      0.77       194

    accuracy                           0.91      1105
   macro avg       0.84      0.88      0.86      1105
weighted avg       0.92      0.91      0.91      1105

Fine 1
              precision    recall  f1-score   support

           0       0.92      0.90      0.91       974
           1       0.38      0.45      0.41       131

    accuracy                           0.85      1105
   macro avg       0.65      0.68      0.66      1105
weighted avg       0.86      0.85      0.85      110

In [23]:
acc/total

NameError: name 'acc' is not defined

In [ ]:
final_fine = [[],[],[],[]]

In [ ]:
final_fine